# Neural Network creation

This script takes results from "aggregated results", and generates neural
networks based on them.


In [ ]:
import sys, time, datetime
from datetime import datetime as dt
t_init = time.time()

In [ ]:
import sys
from utils.setup import SetupNeuralNetworks

argv  = sys.argv[1:]
# argv  = ["-c", "./nn_config/cfg_testing.yml"]
# argv  = ["-c", "./nn_config/cfg_NN_Creation.yml"]
# argv  = ["-c", "./nn_config/cfg_NN_Creation_rasp.yml"]
# argv  = ["-c", "./nn_config/cfg_NN_Creation_climate_invariant.yml"]
# argv  = ["-c", "./nn_config/cfg_NN_Creation_rasp_tphystnd_1-5.yml"]
# argv  = ["-c", "./nn_config/cfg_NN_Creation_climate_invariant_tphystnd_26-30.yml"]
# argv  = ["-c", "./nn_config/220224_threshold_optimization/single/cfg_NN_Creation_rasp_tphystnd_28.yml"]
# argv  = ["-c", "./nn_config/220224_threshold_optimization/pdf/cfg_NN_Creation_rasp_parcorr_pdf-0.59_phq_24.yml"]

setup = SetupNeuralNetworks(argv)


One can check both, attributes and methods in **setup** by, for example:\
**dir(setup)\
setup.__dict__** # setup.__dict__.keys() & setup.__dict__.values()

# Horovod

In [ ]:
import math
import tensorflow as tf
import horovod.tensorflow.keras as hvd

# Horovod: initialize Horovod.
hvd.init()

# Horovod: pin GPU to be used to process local rank (one GPU per process)
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')
#     for gpu in gpus:
#         tf.config.experimental.set_memory_growth(gpu, True)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()],'GPU')

# Flag to indicate whether this is the MPI root
# is_root = hvd.rank() == 0

# # Horovod: adjust number of epochs based on number of GPUs.
# epochs = int(math.ceil(setup.epochs / hvd.size()))
# print('\n')
# print(f'hvd.size: {hvd.size()}')
# print(f'epochs: {setup.epochs}')
# print('\n')

# Horovod: broadcast initial variable states from rank 0 to all other processes.
# This is necessary to ensure consistent initialization of all workers when
# training is started with random weights or restored from a checkpoint.
setup.horovod = hvd.callbacks.BroadcastGlobalVariablesCallback(0)


# Create Neural Networks

In [ ]:
from neural_networks.models_parallelization import generate_models
model_descriptions = generate_models(setup)

**model_descriptions** is a list with all models to be trained. Note, however, that each element in model_description is a class obj. and one can check both, attributes and methods by for example:\
dir(dir(model_descriptions[0]))\
model_descriptions[0].__dict__** # model_descriptions[0].__dict__.keys() & model_descriptions[0].__dict__.values()

# Training

Train all models in the model list and store the results.

In [ ]:
from neural_networks.training_parallelization import train_all_models

train_all_models(model_descriptions, setup)

In [ ]:
t_total = datetime.timedelta(
    seconds=time.time() - t_init
)
print(f"{dt.now()} Finished. Time: {t_total}")